<a href="https://colab.research.google.com/github/MihaiDogariu/CV3/blob/main/laborator/CV%203%20-%20Lab%20%2310.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Generative Adversarial Network
Rețelele generative adversariale au scopul de genera eșantioane noi, nemaiîntâlnite în baza de date de antrenare. Pentru a realiza acest lucru, 2 rețele diferite (generator-G și discriminator-D) sunt puse în competiție, una împotriva celeilalte:
- Discriminatorul are rolul de a determina dacă un eșantion pe care îl "vede" provine din baza de date cu exemple reale sau dacă a fost sintetizat de către generator. 
- Generatorul are rolul de a crea eșantioane care să păcălească discriminatorul, acesta clasificându-le ca fiind reale.

<div>
  <center>
    <img src="https://drive.google.com/uc?export=view&id=1_nqS1h64r_txvmL13TRNeepz3eKODEYx" width="400" class="center">
    <p>Figura 1. Structura generală a unui GAN.</p>
  </center>
</div>

În acest sens, întregul ansamblu este antrenat pentru a optimiza funcția mini-max:

$\displaystyle\min_{\theta}\max_{\phi}V(G_{\theta},D_{\phi})=\mathbb{E}_{x\sim p_{data}}[\log D_{\phi}(x)] + \mathbb{E}_{z\sim p_{z}}[\log (1 - D_{\phi}(G_{\theta}(z)))]$

Laboratorul curent se axează pe antrenarea unui GAN pentru a genera fețe de persoane pornind de la baza de date CelebA.

TODO:
1. Generati doi vectori latenti diferiti si interpolarea dintre ei in 20 de pasi. Rulati generatorul pe toti acesti vectori si observati trecerea de la o imagine la cealalta, ca urmare a interpolarii in spatiul latent.
1. Generati un vector latent. Modificati, pe rand, doar cate o componenta a acestui vector latent si observati diferenta care apare in spatiul imaginilor intre imaginea generata cu vectorul latent original si vectorul latent alterat.
1. Generati 100 vectori latenti si afisati-i intr-o matrice 10 x 10. Selectati 3 grupuri a cate 3 vectori care va genereaza imagini cu caracteristici similare (e.g. 3 vectori care genereaza femei cu par blond, 3 vectori care genereaza barbati cu barba etc.). Gasiti media fiecarui set de cate 3 vectori si realizati operatii aritmetice intre mediile respective asemanator slide-ului 76 din modulul M4.

In [ ]:
import random
import torch

from torchvision import transforms, datasets

import os
import zipfile

import gdown

from torch.utils.data import Dataset
from natsort import natsorted
from PIL import Image

import numpy as np
import matplotlib.pyplot as plt
import torchvision.utils as vutils

import torch.nn as nn

import matplotlib.animation as animation
from IPython.display import HTML

In [ ]:
manual_seed = 999
random.seed(manual_seed)
torch.manual_seed(manual_seed)

## Descarcarea bazei de date

In [ ]:
!mkdir data_faces && wget https://s3-us-west-1.amazonaws.com/udacity-dlnfd/datasets/celeba.zip

with zipfile.ZipFile("celeba.zip","r") as zip_ref:
  zip_ref.extractall("data_faces/")

data_root = 'data_faces'
img_folder = 'data_faces/img_align_celeba'
img_list = os.listdir(img_folder)
print("Total imagini: {}".format(len(img_list)))

## Clasa pentru a prelucra baza de date CelebA

In [ ]:
class CelebADataset(Dataset):
  def __init__(self, root_dir, transform=None):
    # Citire imagini din root_dir
    image_names = os.listdir(root_dir)

    self.root_dir = root_dir
    self.transform = transform 
    self.image_names = natsorted(image_names)

  def __len__(self): 
    return len(self.image_names)

  def __getitem__(self, idx): # functie necesara pentru dataloader
    img_path = os.path.join(self.root_dir, self.image_names[idx])
    img = Image.open(img_path).convert('RGB')
    if self.transform:
      img = self.transform(img)

    return img

## Pregatirea bazei de date

In [ ]:
# Dimensiunea la care vor fi redimensionate imaginile
image_size = 64
# Transformarile aplicate pe baza de date
transform=transforms.Compose([
    transforms.Resize(image_size),
    transforms.CenterCrop(image_size),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5],
                          std=[0.5, 0.5, 0.5])
])

# Incarcarea bazei de date si aplicarea transformarilor
celeba_dataset = CelebADataset(f'{img_folder}', transform)

## Setarea dataloader-ului

In [ ]:
# Setarea parametrilor pentru dataloader
ngpu = 1
device = torch.device('cuda:0' if (
    torch.cuda.is_available() and ngpu > 0) else 'cpu')

batch_size = 128
num_workers = 0 if device.type == 'cuda' else 2
pin_memory = True if device.type == 'cuda' else False

celeba_dataloader = torch.utils.data.DataLoader(celeba_dataset,
                                                batch_size=batch_size,
                                                num_workers=num_workers,
                                                pin_memory=pin_memory,
                                                shuffle=True)

## Inspectia vizuala a bazei de date

In [ ]:
# Exemplu de imagini din baza de date de antrenare
real_batch = next(iter(celeba_dataloader))
image_grid = vutils.make_grid(real_batch.to(device)[:64],
                              padding=2,
                              normalize=True).cpu()
image_grid = np.transpose(image_grid, (1, 2, 0))

plt.figure(figsize=(8, 8))
plt.axis('off')
plt.title('Training Images')
plt.imshow(image_grid)

## Functie pentru a initializa ponderile retelelor

In [ ]:
def weights_init(m):
    if type(m) == nn.Conv2d or type(m) == nn.ConvTranspose2d:
        nn.init.normal_(m.weight.data, 0.0, 0.02)
    elif type(m) == nn.BatchNorm2d:
        nn.init.normal_(m.weight.data, 1.0, 0.02)
        nn.init.constant_(m.bias.data, 0)

## Crearea blocurilor componente arhitecturii DCGAN
Acestea se bazeaza pe straturi uzuale, insa pot fi modularizate in acest fel pentru a reduce codul aferent modelelor propriu-zise

In [ ]:
# Blocuri utilizate in reteaua DCGAN
class ConvTransposeBlock(nn.Module):

    def __init__(self,
                 in_channels,
                 out_channels,
                 kernel_size,
                 stride,
                 padding,
                 bias=False,
                 inplace=True):
        super(ConvTransposeBlock, self).__init__()

        self.layers = nn.Sequential(
            nn.ConvTranspose2d(in_channels,
                               out_channels,
                               kernel_size,
                               stride,
                               padding,
                               bias=bias), nn.BatchNorm2d(out_channels),
            nn.LeakyReLU(0.2, inplace=inplace))

    def forward(self, x):
        return self.layers(x)

class ConvBlock(nn.Module):

    def __init__(self,
                 in_channels,
                 out_channels,
                 kernel_size,
                 stride,
                 padding,
                 bias=False,
                 inplace=True,
                 use_batch_norm=True):
        super(ConvBlock, self).__init__()

        layers = nn.ModuleList()
        layers.append(
            nn.Conv2d(in_channels,
                      out_channels,
                      kernel_size,
                      stride,
                      padding,
                      bias=bias))
        if use_batch_norm:
            layers.append(nn.BatchNorm2d(out_channels))
        layers.append(nn.LeakyReLU(0.2, inplace=inplace))

        self.layers = nn.Sequential(*layers)

    def forward(self, x):
        return self.layers(x)

## Crearea modelului generatorului

In [ ]:
class Generator(nn.Module):

    def __init__(self, latent_dim, base_channels, out_channels):
        super(Generator, self).__init__()

        self.layers = nn.Sequential(
            # Intrarea este vectorul de zgomot, z
            ConvTransposeBlock(latent_dim, base_channels * 8, 4, 1,
                               0),  # base_channels*8 x 4 x 4
            ConvTransposeBlock(base_channels * 8, base_channels * 4, 4, 2,
                               1),  # base_channels*4 x 8 x 8
            ConvTransposeBlock(base_channels * 4, base_channels * 2, 4, 2,
                               1),  # base_channels*2 x 16 x 16
            ConvTransposeBlock(base_channels * 2, base_channels, 4, 2,
                               1),  # base_channels x 32 x 32
            nn.ConvTranspose2d(base_channels, out_channels, 4, 2, 1,
                               bias=False),  # out_channels x 64 x 64
            nn.Tanh())

    def forward(self, x):
        return self.layers(x)

## Crearea modelului discriminatorului

In [ ]:
class Discriminator(nn.Module):

    def __init__(self, base_channels, in_channels):
        super(Discriminator, self).__init__()

        self.layers = nn.Sequential(
            # Dimensiunea intrarii este in_channels x 64 x 64
            ConvBlock(in_channels, base_channels, 4, 2,
                      1),  # base_channels x 32 x 32
            ConvBlock(base_channels, base_channels * 2, 4, 2,
                      1),  # base_channels*2 x 16 x 16
            ConvBlock(base_channels * 2, base_channels * 4, 4, 2,
                      1),  # base_channels*4 x 8 x 8
            ConvBlock(base_channels * 4, base_channels * 8, 4, 2,
                      1),  # base_channels*8 x 4 x 4
            nn.Conv2d(base_channels * 8, 1, 4, 1, 0, bias=False),
            nn.Sigmoid())

    def forward(self, x):
        return self.layers(x)

## Pregatirea generatorului

In [ ]:
# Dimensiunea vectorului de zgomot
latent_dim = 100
# Numarul de canale ale vectorului de trasaturi al generatorului
generator_channels = 64
# Numarul de canale ale imaginilor de antrenare
image_channels = 3

# Crearea generatorului
generator = Generator(latent_dim, generator_channels, image_channels).to(device)
if (device.type == 'cuda' and ngpu > 0):
    generator = nn.DataParallel(generator, list(range(ngpu)))
generator.apply(weights_init)

generator

## Pregatirea discriminatorului

In [ ]:
# Numarul de canale ale vectorului de trasaturi al discriminatorului
discriminator_channels = 64

# Crearea discriminatorului
discriminator = Discriminator(discriminator_channels, image_channels)
if (device.type == 'cuda' and ngpu > 0):
    discriminator = nn.DataParallel(discriminator, list(range(ngpu)))
discriminator.apply(weights_init)

discriminator

## Setarea hiperparametrilor, a pierderii si a optimizatorului

In [ ]:
# Hiperparametri
lr = 0.0002
beta1 = 0.5
beta2 = 0.999
num_epochs = 10

# Functia de pierdere de cross-entropie binara, utilizata pentru a evalua discriminatorul
criterion = nn.BCELoss()

# Optimizatorul pentru generator
generator_optim = torch.optim.Adam(generator.parameters(),
                                   lr=lr,
                                   betas=(beta1, beta2))
# Optimizatorul pentru discriminator
discriminator_optim = torch.optim.Adam(discriminator.parameters(),
                                       lr=lr,
                                       betas=(beta1, beta2))

## Pregatirea componentelor ce vor urmari evolutia antrenarii

In [ ]:
# Se va folosi un batch de vectori de zgomot pentru a urmari evolutia antrenarii
fixed_latent_vectors = torch.randn((64, latent_dim, 1, 1), device=device)

# O lista cu imaginile generate de-a lungul antrenarii
img_list = []

generator_losses = []
discriminator_losses = []
# Retinem numarul de iteratii/batch-uri procesate la fiecare epoca
iters = 0

## Antrenarea propriu-zisa

In [ ]:
print('Starting training loop...')

for epoch in range(num_epochs):
    for i, real_batch in enumerate(celeba_dataloader):
        # ----------------- Antrenarea Discriminatorului ----------------------
        # Scopul antrenarii discriminatorului este de a maximiza log(D(x)) + log(1 - D(G(z)))
        
        # --- Pasul 1. Procesam un batch de date reale ---
        # Resetam gradientul acumulat la pasul anterior
        discriminator.zero_grad()

        # Extragem dimensiunea batch-ului, deoarece ultimul batch poate avea o 
        # dimensiune diferita fata de cea stabilita ca hiperparametru, in cazul 
        # in care baza de date nu are un nr de exemple multiplu intreg de batch_size
        batch_size = real_batch.size(0)

        # Se pot folosi etichete soft, in loc de clasicele 0/1 pentru a avea un
        # comportament mai lin al functiei de pierdere.
        # In loc de 1, folosim valori aleatoare intre 0.7 si 1.2
        real_labels = (1.2 - 0.7) * torch.rand((batch_size,)) + 0.7
        real_labels = real_labels.to(device)
        real_batch = real_batch.to(device)

        # Prelucram batch-ul de date reale cu discriminatorul
        output = discriminator(real_batch).view(-1)

        # Calculam functia de pierdere si gradientul
        discriminator_real_loss = criterion(output, real_labels)
        discriminator_real_loss.backward()
        
        # Retinem iesirea pentru a o afisa pe ecran
        D_x = output.mean().item()

        
        # --- Pasul 2. Procesam un batch de date sintetice ---
        # Procesarea batch-ului de date sintetice se face mai intai prin generator
        # si apoi prin discriminator, D(G(z))
        
        # Generam un vector de zgomot z
        latent_vectors_batch = torch.randn((batch_size, latent_dim, 1, 1),
                                           device=device)
        # Generam imagini false cu generatorul
        fake_batch = generator(latent_vectors_batch)

        # Folosim etichete soft, ca in cazul datelor reale
        # In loc de 0, folosim valori aleatoare intre 0.0 si 0.3
        fake_labels = (0.3 - 0.0) * torch.rand((batch_size,)) + 0.0
        fake_labels = fake_labels.to(device)

        # Prelucram batch-ul de date sintetice cu discriminatorul
        output = discriminator(fake_batch.detach()).view(-1)

        # Calculam functia de pierdere si gradientul
        discriminator_fake_loss = criterion(output, fake_labels)
        discriminator_fake_loss.backward()

        # Retinem iesirea pentru a o afisa pe ecran: D(G(z1))
        D_G_z1 = discriminator_fake_loss.mean().item()
        
        # Calculam pierderea totala a discriminatorului
        discriminator_loss = discriminator_real_loss + discriminator_fake_loss

        # Actualizam ponderile discriminatorului
        discriminator_optim.step()
        # ---------------- Sfarsit Antrenare Discriminator ---------------------



        # ------------------- Antrenarea Generatorului -------------------------
        # Scopul antrenarii discriminatorului este de a maximiza log(D(G(z)))
        
        # Resetam gradientul acumulat la pasul anterior
        generator.zero_grad()
        
        # Procesam prin discriminator batch-ul de date sintetice generat 
        # la pasul anterior (la antrenarea discriminatorului)
        output = discriminator(fake_batch).view(-1)

        # Calculam functia de pierdere a generatorului
        # In acest caz, generatorul vede toate iesirile sale ca fiind reale, 
        # deci vom interschimba etichetele sintetice cu cele reale pentru a 
        # calcula aceasta functie de pierdere
        generator_loss = criterion(output, real_labels)

        # Calculam gradientul
        generator_loss.backward()

        # Actualizam ponderile generatorului
        generator_optim.step()

        # Retinem iesirea pentru a o afisa pe ecran: D(G(z1))
         D_G_z2 = output.mean().item()

        # Afisam caracteristicile antrenarii la fiecare 50 iteratii
        if i % 50 == 0:
            print(
                f'[{epoch}/{num_epochs - 1}][{i}/{len(celeba_dataloader)}]\t Loss_D:{discriminator_loss.item():4f} Loss_G:{generator_loss.item():4f} D(x):{D_x:4f} D(G(z)):{D_G_z1:4f}/{D_G_z2:4f}'
            )

        # Retinem pierderile pentru a le afisa pe grafic
        discriminator_losses.append(discriminator_loss.item())
        generator_losses.append(generator_loss.item())

        # La fiecare 500 de iteratii vom procesa din nou vectorul de zgomot fix,
        # generat inainte de antrenare. In felul acesta, vom urmari evolutia 
        # generatorului pe acelasi vector de zgomot.
        if (iters % 500 == 0) or ((epoch == num_epochs - 1) and
                                  (i == len(celeba_dataloader) - 1)):
            with torch.no_grad():
                fixed_generations = generator(
                    fixed_latent_vectors).detach().cpu()
            img_list.append(
                vutils.make_grid(fixed_generations, padding=2, normalize=True))

        # Incrementam iteratiile
        iters += 1

## Evaluare cantitativa

In [ ]:
# Afisam pierderile generatorului si ale discriminatorului pe grafic
plt.figure(figsize=(10, 5))
plt.title("Generator and Discriminator Loss During Training")
plt.plot(generator_losses, label="G")
plt.plot(discriminator_losses, label="D")
plt.xlabel("iterations")
plt.ylabel("Loss")
plt.legend()
plt.show()

## Evaluare calitativa

In [ ]:
fig = plt.figure(figsize=(8, 8))
plt.axis("off")
ims = [[plt.imshow(np.transpose(i, (1, 2, 0)), animated=True)] for i in img_list
      ]
ani = animation.ArtistAnimation(fig,
                                ims,
                                interval=1000,
                                repeat_delay=1000,
                                blit=True)

HTML(ani.to_jshtml())

In [ ]:
# Dupa antrenarea completa a generatorului si a discriminatorului afisam in
# paralel datele reale si cele sintetice pentru a urmari diferentele calitative
# Extragem un batch de date reale
real_batch = next(iter(celeba_dataloader))

# Afisam un batch de date reale
plt.figure(figsize=(15,15))
plt.subplot(1,2,1)
plt.axis("off")
plt.title("Real Images")
plt.imshow(np.transpose(vutils.make_grid(real_batch.to(device)[:64], padding=5, normalize=True).cpu(),(1,2,0)))

# Afisam un batch de date sintetizate la ultima epoca
plt.subplot(1,2,2)
plt.axis("off")
plt.title("Fake Images")
plt.imshow(np.transpose(img_list[-1],(1,2,0)))
plt.show()